In [41]:
from kth_dataset import KTH_Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from LSTMscratch import LSTMscratch
import torch
import torch.nn as nn
from Model_Wrapper import Wrapper
import helper 


In [42]:
helper.set_random_seed()


Random seed set to: 13


In [ ]:
# Create a dataset and a DataLoader
transforms1 = { 'train':transforms.Compose([
        transforms.ToTensor(),  # Convert image to tensor
        transforms.Resize([32, 32]), 
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1))
    ]),
    'val':transforms.Compose([
        transforms.ToTensor(),  # Convert image to tensor
        transforms.Resize([32, 32]), 
    ])}
# Initialize dataset
test_dataset = KTH_Dataset(root_dir="/home/nfs/inf6/data/datasets/kth_actions/processed", sequence_length=15,split="test", transform=transforms1,use_saved_samples=False)
train_dataset = KTH_Dataset(root_dir="/home/nfs/inf6/data/datasets/kth_actions/processed", sequence_length=15,split="train", transform=transforms1,use_saved_samples=False)


In [44]:
train_dataset.__len__()

181826

In [45]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,num_workers=8,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False,num_workers=8,pin_memory=True)
print("test_dataset per epoch",next(iter(test_loader))[0].size())

test_dataset per epoch torch.Size([16, 15, 1, 32, 32])


In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [47]:
model = LSTMscratch(input_dim=16, hidden_dim=16, number_of_layers=2,device=device)
model_1 = LSTMscratch(input_dim=16, hidden_dim=16, number_of_layers=2,device=device,use_custom=False)

In [48]:
# model_t = SequentialClassifierWithCells(emb_dim=128, hidden_dim=128, num_layers=2, mode="zeros")
model = model.to(device)
model_1 = model_1.to(device)


In [49]:
# classification loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 5 epochs
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [50]:
classes = ["Boxing","HandClapping","HandWaving","Jogging","Running","Walking"]
writer = helper.new_writer("models","LSTMscratch")
train = Wrapper(model_name="LSTM", model = model, device = device, criterion = criterion, optimizer = optimizer,writer=writer,show_progress_bar= True)
train.train(10,train_loader,test_loader,classes)
writer.close()
train.valid_accuracy()
train.plot_loss_acc()


  0%|          | 0/11365 [00:00<?, ?it/s]

Epoch 1 Iter 11365: loss 1.68365.: 100%|██████████| 11365/11365 [08:00<00:00, 23.64it/s]

Accuracy/train 29.18289118045081
Loss/train 1.4135669457508613


KeyboardInterrupt: 

In [76]:
writer = helper.new_writer("models","LSTM")
train = Wrapper(model_name="LSTMcell", model = model_1, device = device, criterion = criterion, optimizer = optimizer,writer=writer,show_progress_bar= True)
train.train(10,train_loader,test_loader,classes)
writer.close()
train.valid_accuracy()
train.plot_loss_acc()


Epoch 1 Iter 10527: loss 1.76939.: 100%|██████████| 10527/10527 [15:12<00:00, 11.54it/s]

Accuracy/train 23.512760076486128
Loss/train 1.799820581766188


Accuracy/test 23.434916905942746
Loss/test 1.7952643549917302


Epoch 2 Iter 10527: loss 1.83568.: 100%|██████████| 10527/10527 [15:11<00:00, 11.55it/s]

Accuracy/train 23.471734336132613
Loss/train 1.7998249094359633


Accuracy/test 23.434916905942746
Loss/test 1.7952643549917302


Epoch 3 Iter 10527: loss 1.69221.: 100%|██████████| 10527/10527 [15:13<00:00, 11.53it/s]

Accuracy/train 23.36618330929439
Loss/train 1.7998155394521795


KeyboardInterrupt: 